In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [62]:
data = pd.read_csv('./data/queried/transactions_1.csv', index_col=0)
data.head()

,address from,address to,timestamp,amount,gasUsed,contractInteracted
flag,,,,,,
1,0x6ff0aa6a46332e3cfa80cb8c5b53531307c01043,0x15b4e32d5026a5a9ad3b68becadceb00b4eb9729,1521174927,200000000000000000,21000,NaN
1,0x564286362092d8e7936f0549571a803b203aaced,0x15b4e32d5026a5a9ad3b68becadceb00b4eb9729,1521176311,190000000000000000,21000,NaN
1,0x54092cb3badd6d3c4bd85a37400176afe0ff6905,0x15b4e32d5026a5a9ad3b68becadceb00b4eb9729,1521279093,100000000000000,21000,NaN
1,0xd551234ae421e3bcba99a0da6d736074f22192ff,0x15b4e32d5026a5a9ad3b68becadceb00b4eb9729,1521374468,2500000000000000000,21000,NaN
1,0x21675eda32bd424bc6522b1f2ace7d9282d20f51,0x15b4e32d5026a5a9ad3b68becadceb00b4eb9729,1521638799,226000000000000000,21000,NaN


In [63]:
dfs = []

for i in range(20):  
    file_path = f'../bt4012-group13/data/queried/transactions_{i}.csv'
    dfs.append(pd.read_csv(file_path))


merged_df = pd.concat(dfs)

# Display the shape and first few rows of the merged dataframe
merged_df.shape, merged_df.head()

/var/folders/03/rh1df1yd5y7bb1shd2_734pw0000gn/T/ipykernel_46190/2361987607.py:5: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(file_path))


((5238249, 7),
    flag                                address from  \
 0     1  0xc6c198c7e472de4313f2ae9c9b02185dd8d91262   
 1     1  0x028d61f42f1ca1ad6887f3311a0b1789ef6dadbb   
 2     1  0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98   
 3     1  0x2c2b7579e86363ccd46d0942f7fe274ccdbae351   
 4     1  0xabf1bbefc50800427ac69fa2e62ceac9b3b11d5b   
 
                                    address to   timestamp  \
 0  0x000000000532b45f47779fce440748893b257865  1594562749   
 1  0x000000000532b45f47779fce440748893b257865  1594563733   
 2  0x000000000532b45f47779fce440748893b257865  1594563793   
 3  0x000000000532b45f47779fce440748893b257865  1594563893   
 4  0x000000000532b45f47779fce440748893b257865  1594564904   
 
                  amount  gasUsed contractInteracted  
 0     10000000000000000    21000                NaN  
 1  10000000000000000000    21000                NaN  
 2   5000000000000000000    21000                NaN  
 3   5000000000000000000    21000                NaN 

In [64]:
data = pd.DataFrame(merged_df)
data = data.drop(['Unnamed: 0'], axis=1)
data.head()


KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
len(data)

5238249

In [ ]:
# Convert UNIX timestamp to datetime format and extract features
data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour

# Drop the original timestamp and date columns
data.drop(columns=['timestamp', 'date'], inplace=True)

# Fill missing values in 'contractInteracted' with a placeholder
data['contractInteracted'].fillna('None', inplace=True)

# Display the modified dataframe
data.head()


,flag,address from,address to,amount,gasUsed,contractInteracted,year,month,day,hour
0,1,0xc6c198c7e472de4313f2ae9c9b02185dd8d91262,0x000000000532b45f47779fce440748893b257865,10000000000000000,21000,None,2020,7,12,14
1,1,0x028d61f42f1ca1ad6887f3311a0b1789ef6dadbb,0x000000000532b45f47779fce440748893b257865,10000000000000000000,21000,None,2020,7,12,14
2,1,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x000000000532b45f47779fce440748893b257865,5000000000000000000,21000,None,2020,7,12,14
3,1,0x2c2b7579e86363ccd46d0942f7fe274ccdbae351,0x000000000532b45f47779fce440748893b257865,5000000000000000000,21000,None,2020,7,12,14
4,1,0xabf1bbefc50800427ac69fa2e62ceac9b3b11d5b,0x000000000532b45f47779fce440748893b257865,535414469040000000,21000,None,2020,7,12,14


In [ ]:
# Define the feature columns
# List of feature column names
feature_cols = ['address from', 'address to', 'amount', 'year', 'month', 'day', 'hour', 'contractInteracted']
feature = data[feature_cols]
target = data['flag']
feature.head()


,address from,address to,amount,year,month,day,hour,contractInteracted
0,0xc6c198c7e472de4313f2ae9c9b02185dd8d91262,0x000000000532b45f47779fce440748893b257865,10000000000000000,2020,7,12,14,None
1,0x028d61f42f1ca1ad6887f3311a0b1789ef6dadbb,0x000000000532b45f47779fce440748893b257865,10000000000000000000,2020,7,12,14,None
2,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x000000000532b45f47779fce440748893b257865,5000000000000000000,2020,7,12,14,None
3,0x2c2b7579e86363ccd46d0942f7fe274ccdbae351,0x000000000532b45f47779fce440748893b257865,5000000000000000000,2020,7,12,14,None
4,0xabf1bbefc50800427ac69fa2e62ceac9b3b11d5b,0x000000000532b45f47779fce440748893b257865,535414469040000000,2020,7,12,14,None


In [ ]:
feature['to'] = feature['address to'] + feature['contractInteracted']
feature_new = feature.drop(['address to', 'contractInteracted'], axis=1)

/var/folders/03/rh1df1yd5y7bb1shd2_734pw0000gn/T/ipykernel_46190/3948027829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature['to'] = feature['address to'] + feature['contractInteracted']


In [ ]:
feature_new.head(20)

,address from,amount,year,month,day,hour,to
0,0xc6c198c7e472de4313f2ae9c9b02185dd8d91262,10000000000000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
1,0x028d61f42f1ca1ad6887f3311a0b1789ef6dadbb,10000000000000000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
2,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,5000000000000000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
3,0x2c2b7579e86363ccd46d0942f7fe274ccdbae351,5000000000000000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
4,0xabf1bbefc50800427ac69fa2e62ceac9b3b11d5b,535414469040000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
5,0xcf1d46459a642f2c2a9fbd6877c87b9d8976305a,4019689281920000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
6,0x7c07bad26409c6c949714a9471e918c522879f9d,444896249040000000,2020,7,12,14,0x000000000532b45f47779fce440748893b257865None
7,0xe4504cf4ee8acafb6ee5dc1c57fbf6fb41bccfe2,9999601020000000000,2020,7,12,23,0x000000000532b45f47779fce440748893b257865None
8,0xf1aa3c61c0e7f2436b035526fecda98a729d6934,5000000000000000000,2020,7,13,3,0x000000000532b45f47779fce440748893b257865None
9,0xed292262788b9eaed8bb3739715c5db0d6522d3c,10000000000000000000,2020,7,13,3,0x000000000532b45f47779fce440748893b257865None


In [65]:
# loading processed data
kaggle = pd.read_csv('./data/processed/kaggle_transaction_dataset.csv')
kaggle.head()

,address,flag,avgTimeBetweenSentTnx,avgTimeBetweenRecTnx,lifetime,sentTransactions,receivedTransactions,createdContracts,numUniqRecAddress,numUniqSentAddress,...,maxValReceived,avgValReceived,minValSent,maxValSent,avgValSent,totalTransactions,totalEtherSent,totalEtherReceived,totalEtherSentContracts,totalEtherBalance
0,0xd0cc2b24980cbcca47ef755da88b220a82291407,1,5.807655e+04,7.148260e+07,46.0,66,21,0,17,47,...,266.000000,17.486340,0.0,183.0,5.559412,87,366.92119,367.213130,0.0,0.291940
1,0x4cdc1cba0aeb5539f2e0ba158281e67e0e54a9b1,1,0.000000e+00,7.529115e+08,35.0,0,2,0,2,0,...,0.010000,0.006771,0.0,0.0,0.000000,2,0.00000,0.013541,0.0,0.013541
2,0x00e01a648ff41346cdeb873182383333d2184dd1,1,1.710279e+06,6.820612e+07,299.0,15,22,0,21,13,...,49.999314,9.100750,0.0,25.0,13.330000,37,199.95000,200.216494,0.0,0.266494
3,0x858457daa7e087ad74cdeeceab8419079bc2ca03,1,1.576106e+04,1.352571e+07,36.0,109,111,0,104,54,...,193.500000,14.689887,0.0,680.0,14.959358,220,1630.57000,1630.577419,0.0,0.007419
4,0x240e125c20a4cc84bd6e7f8d1fd07aff4c06d43d,1,0.000000e+00,3.751409e+08,82.0,1,4,0,4,1,...,1.000000,0.475550,1.9,1.9,1.900000,5,1.90000,1.902199,0.0,0.002199


In [66]:
kaggle.describe()

,flag,avgTimeBetweenSentTnx,avgTimeBetweenRecTnx,lifetime,sentTransactions,receivedTransactions,createdContracts,numUniqRecAddress,numUniqSentAddress,minValReceived,maxValReceived,avgValReceived,minValSent,maxValSent,avgValSent,totalTransactions,totalEtherSent,totalEtherReceived,totalEtherSentContracts,totalEtherBalance
count,21987.000000,2.198700e+04,2.198700e+04,2.198700e+04,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,21987.000000,2.198700e+04,2.198700e+04,21987.000000,2.198700e+04
mean,0.333333,2.017807e+06,8.849104e+07,9.782811e+04,337.691409,448.022786,2.641788,167.290535,66.613408,27.203154,380.656260,73.831433,5.603368,233.015954,26.741914,788.355983,6.238740e+03,6.824447e+03,0.002042,5.857052e+02
std,0.471415,3.068441e+07,1.905694e+08,2.417469e+05,1496.171520,1807.664973,122.922835,868.424851,496.873942,427.784109,10626.929063,2767.424229,206.194426,6315.603885,272.599897,2405.265091,2.494383e+05,2.475040e+05,0.263258,1.694017e+05
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,-1.560535e+07
25%,0.000000,0.000000e+00,1.020000e+00,2.300000e+01,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,0.300000,0.099454,0.000000,0.048303,0.008546,4.000000,7.312695e-02,8.111346e-01,0.000000,0.000000e+00
50%,0.000000,2.171600e+02,1.033698e+04,2.150000e+02,3.000000,8.000000,0.000000,3.000000,2.000000,0.011846,2.000000,0.624387,0.000000,1.991536,0.440000,18.000000,5.167382e+00,8.870386e+00,0.000000,1.638000e-03
75%,1.000000,3.269873e+04,6.604927e+07,1.716344e+04,32.000000,42.000000,0.000000,10.000000,8.000000,0.300000,14.934265,3.322822,0.439608,17.409571,4.512919,133.000000,6.222540e+01,7.798907e+01,0.000000,4.201412e-02
max,1.000000,8.291299e+08,8.309932e+08,1.954861e+06,10000.000000,10000.000000,9995.000000,9999.000000,9950.000000,25533.614518,800000.000000,283618.831600,25533.614329,611102.014456,25533.614329,19995.000000,2.858096e+07,2.858159e+07,39.000000,1.428864e+07


In [67]:
data['flag'].value_counts()


0    4878999
1     359250
Name: flag, dtype: int64

In [72]:
feature = kaggle.drop(['flag', 'address'], axis=1)
target = kaggle['flag']

In [73]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15390, 19), (6597, 19), (15390,), (6597,))

In [80]:
# Define the random forest model
rf_model = RandomForestClassifier(random_state=1)

# Fit the model to the training data
rf_model.fit(X_train, y_train)
# make predictions on the test data
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95      4392
           1       0.92      0.86      0.89      2205

    accuracy                           0.93      6597
   macro avg       0.93      0.91      0.92      6597
weighted avg       0.93      0.93      0.93      6597



In [75]:
# training xgboost model
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
# Define the XGBoost model
xgb_model = XGBClassifier(objective='binary:logistic', random_state=1)

# Fit the model to the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9422464756707595
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      4392
           1       0.94      0.88      0.91      2205

    accuracy                           0.94      6597
   macro avg       0.94      0.93      0.93      6597
weighted avg       0.94      0.94      0.94      6597



In [ ]:
# train with oversampling
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
rf_model.fit(X_train_res, y_train_res)
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

In [78]:
# train xgboost with grid search
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2]
}
# Define the XGBoost model
xgb_model = XGBClassifier(objective='binary:logistic', random_state=1)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   4.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   4.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   4.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   4.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=100; total time=   4.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   8.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   8.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   8.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   8.1s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=200; total time=   8.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   6.6s
[CV] END ..learning_rate=0.01, max_depth=4, n_e

In [79]:
# Define the XGBoost model with the best parameters
xgb_model_tuned = XGBClassifier(objective='binary:logistic', random_state=1)
xgb_model_tuned.set_params(**grid_search.best_params_)

# Fit the model to the training data
xgb_model_tuned.fit(X_train, y_train)

# Make predictions on the test data
y_pred = xgb_model_tuned.predict(X_test)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9442170683644081
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4392
           1       0.94      0.89      0.91      2205

    accuracy                           0.94      6597
   macro avg       0.94      0.93      0.94      6597
weighted avg       0.94      0.94      0.94      6597

